## Query 11

Obtener el tiempo promedio de TTFP (tiempo de primera compra) por segmento de clientes.

El tiempo de primera compra se define como la diferencia en días entre la fecha de creación del cliente y la fecha de su primera orden.

Para cada segmento se quiere el promedio en días de TTFP junto con la cantidad de clientes que componen el segmento, la cantidad de clientes que realizaron al menos una compra y la cantidad de clientes que no realizaron ninguna compra.

Para que la query funcione correctamente es necesario que "customers" y "orders" se puedan joinear.

Como customers tiene customer_ids desde 1 a 441895 y orders tiene customer_ids desde 400009 a 500000, podemos restarle 400008 a los customer_ids de orders para que matcheen.

- 1 -> 400009
- 2 -> 400010
- ...

Para resolver esto debemos seguir los siguientes pasos:

1. Importar el dataset de customers (data/raw/customers.csv).
2. Filtrar clientes con datos válidos (customer_id, customer_segment, registration_date).
3. Mapear a (customer_id, (segment, registration_date)).
4. Importar el dataset de orders (data/raw/orders.csv).
5. Filtrar órdenes con datos válidos (order_id, customer_id, created_at).
6. Mapear a (customer_id, order_date) y reducir por customer_id para obtener la primera orden de cada cliente.
7. Hacer left outer join entre clientes y primeras órdenes usando customer_id como clave.
8. Calcular TTFP (días entre registration_date del cliente y primera orden) para clientes que compraron.
9. Agrupar por segmento y calcular: TTFP promedio, total de clientes, clientes con compra, clientes sin compra.
10. Formatear y mostrar resultados.

In [106]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.appName("PrimerasComprasClientes").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

OFFSET = 400008

In [118]:
customers_df = spark.read.csv(
    "../../data/raw/customers.csv", header=True, inferSchema=True
)
customers_rdd = customers_df.rdd

customers_kv = (
    customers_rdd.filter(
        lambda r: r.customer_id and r.customer_segment and r.registration_date
    )
    .map(
        lambda r: (
            str(r.customer_id),
            (
                r.customer_segment.strip().upper(),
                r.registration_date.strip()[:10] if r.registration_date else None,
            ),
        )
    )
    .filter(lambda x: x[1][1] is not None)
    .cache()
)

customers_kv.take(10)

[('1', ('REGULAR', '2022-09-22')),
 ('2', ('BUDGET', '2022-11-14')),
 ('3', ('BUDGET', '2022-10-14')),
 ('4', ('UNDEFINED', '2024-03-16')),
 ('5', ('REGULAR', '2023-07-21')),
 ('6', ('REGULAR', '2023-08-17')),
 ('7', ('PREMIUM', '2025-04-01')),
 ('8', ('REGULAR', '2023-05-05')),
 ('9', ('REGULAR', '2024-04-01')),
 ('10', ('REGULAR', '2022-08-17'))]

In [121]:
orders_df = spark.read.csv("../../data/raw/orders.csv", header=True, inferSchema=True)
orders_rdd = orders_df.rdd

first_orders = (
    orders_rdd.filter(lambda r: r.order_id and r.customer_id and r.created_at)
    .map(
        lambda r: (
            str(r.customer_id),
            r.created_at.strip()[:10] if r.created_at else None,
        )
    )
    .filter(lambda x: x[1] is not None)
    .reduceByKey(lambda a, b: min(a, b))
).cache()

first_orders.take(10)

[('447917', '2023-11-24'),
 ('441229', '2024-11-29'),
 ('442829', '2024-10-08'),
 ('454929', '2023-03-09'),
 ('489351', '2023-05-31'),
 ('416469', '2025-05-20'),
 ('411439', '2022-12-31'),
 ('464064', '2022-11-28'),
 ('468539', '2024-06-21'),
 ('437634', '2025-02-03')]

In [ ]:
def calculate_ttfp_tuple(registration_date, first_order_date):

    try:
        order_date = datetime.strptime(first_order_date, "%Y-%m-%d")
        reg_date = datetime.strptime(registration_date, "%Y-%m-%d")
        if order_date < reg_date:
            return None

        return (order_date - reg_date).days
    except Exception:
        return None


customers_with_orders = (
    customers_kv.leftOuterJoin(
        first_orders
    )  # (customer_id, ((segment, registration_date), first_order_date))
    .map(
        lambda kv: (
            kv[1][0][0],  # segment
            calculate_ttfp_tuple(kv[1][0][1], kv[1][1]) if kv[1][1] else None,  # ttfp
        )
    )
    .map(
        lambda x: (
            x[0],
            (
                x[1],
                1 if x[1] is not None else 0,  # has_first_order
                1,
            ),
        )
    )
    .cache()
)

customers_with_orders.take(10)

[('UNDEFINED', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('UNDEFINED', (None, 0, 1)),
 ('REGULAR', (None, 0, 1)),
 ('REGULAR', (None, 0, 1))]

In [ ]:
segment_stats = (
    customers_with_orders.reduceByKey(
        lambda a, b: (  # (total_ttfp, total_with_first_order, total_customers)
            (a[0] if a[0] is not None else 0) + (b[0] if b[0] is not None else 0),
            a[1] + b[1],
            a[2] + b[2],
        )
    )
    .map(
        lambda kv: (
            kv[0],  # segment
            (kv[1][0] / kv[1][1]) if kv[1][1] > 0 else None,  # avg_ttfp
            kv[1][2],  # total_customers
            kv[1][1],  # total_with_first_order
            kv[1][2] - kv[1][1],  # total_without_first_order
        )
    )
    .collect()
)

In [ ]:
import pandas as pd

if segment_stats:
    results = [
        {
            "segmento": s[0],
            "avg_ttfp": round(s[1], 2) if s[1] is not None else "N/A",
            "total_clientes": s[2],
            "con_primera_compra": s[3],
            "sin_primera_compra": s[4],
        }
        for s in segment_stats
    ]
    df = pd.DataFrame(results).sort_values("segmento")
    display(df)
else:
    print("No results")

,segmento,ttfp_promedio_dias,total_clientes,clientes_con_compra,clientes_sin_compra
2,BUDGET,0,84972,0,84972
3,PREMIUM,0,86042,0,86042
0,REGULAR,0,256735,0,256735
1,UNDEFINED,0,14146,0,14146
